## How to Forecast Time Series Data using any Supervised Learning Model

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mturk24/blog_posts/blob/main/time_series_automl/time_series_automl.ipynb)

This notebook delves into enhancing the process of forecasting daily energy consumption levels by transforming a time series dataset into a tabular format using open-source libraries. We explore the application of a popular multiclass classification model and leverage AutoML with cleanlab to significantly boost our out-of-sample accuracy.

At a high level we will:

- Establish a baseline accuracy by fitting a Prophet forecasting model on our time series data
- Convert our time series data into a tabular format by using open-source featurization libraries and then will show that can outperform our Prophet model with a standard multiclass classification (Gradient Boosting) approach by a **67% reduction in prediction error** (increase by 38% raw percentage points in out-of-sample accuracy).
- Use an AutoML solution for multiclass classification **resulted in a 42% reduction in prediction error** (increase by 8% in raw percentage points in out-of-sample accuracy) compared to our Gradient Boosting model and **resulted in a 81% reduction in prediction error** (increase by 46% in raw percentage points in out-of-sample accuracy) compared to our Prophet forecasting model.

## Initialize time series data for Prophet

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None 

data = pd.read_csv('PJME_hourly.csv', parse_dates=['Datetime'], index_col='Datetime')

# Assuming pjme_data is loaded as before
daily_data = data.resample('D').mean() 

# Prepare data for Prophet
daily_data.reset_index(inplace=True)
daily_data.columns = ['ds', 'y']

## Initialize time series data for featurization into a tabular format

In [2]:
from sklearn.model_selection import train_test_split

# Reset the datetime
data["Datetime"] = data.index
data = data.reset_index(drop=True)

# Create copy for multiclass data 
df = data.copy()

# Convert the datetime column
df['Datetime'] = pd.to_datetime(df['Datetime'])  # Adjust the 'datetime' column name as necessary
df = df.sort_values('Datetime').reset_index(drop=True)


# Obtain day and hour
df['Date'] = pd.to_datetime(df['Datetime']).dt.floor('D')  
df['Hour'] = pd.to_datetime(df['Datetime']).dt.hour

# Create multi-index feature df to compute time series features on
features = df.set_index(['Date', 'Hour'])  
features.drop("Datetime", inplace=True, axis=1)

# Split the data into training and testing sets, respecting the temporal order
X_train, X_test, y_train, y_test = train_test_split(features, features["PJME_MW"], test_size=0.2, shuffle=False)

# Get group lengths
train_lengths = X_train.groupby(level=0).size()
test_lengths = X_test.groupby(level=0).size()

# Obtain common length value for train/test data
train_common_length = train_lengths.mode().iloc[0]
test_common_length = test_lengths.mode().iloc[0]

# Filter train/test data to groups with same common length for featurizer
X_train = X_train.groupby(level=0).filter(lambda x: len(x) == train_common_length)
X_test = X_test.groupby(level=0).filter(lambda x: len(x) == test_common_length)

# Create quartiles based on training data to avoid leakage
quartiles = [X_train['PJME_MW'].quantile(q) for q in [0.25, 0.50, 0.75]]

## Train and Evaluate Prophet Forecasting Model

In [3]:
# Cutoff date at 2015-04-09
cutoff_index = int(len(daily_data) * 0.8)

# Use 80% of data for training set and 20% for test set
train_df = daily_data.iloc[:cutoff_index]
test_df = daily_data.iloc[cutoff_index:]

print("Training Set Shape:", train_df.shape)
print("Testing Set Shape:", test_df.shape)

Training Set Shape: (4847, 2)
Testing Set Shape: (1212, 2)


In [4]:
train_df.tail()

,ds,y
4842,2015-04-05,24577.500000
4843,2015-04-06,26996.666667
4844,2015-04-07,27177.833333
4845,2015-04-08,29136.041667
4846,2015-04-09,30535.291667


In [5]:
test_df.head()

,ds,y
4847,2015-04-10,29190.166667
4848,2015-04-11,24774.291667
4849,2015-04-12,24407.625000
4850,2015-04-13,26825.333333
4851,2015-04-14,26952.125000


In [12]:
import numpy as np
from prophet import Prophet
from sklearn.metrics import accuracy_score

# Initialize model and train it on training data
model = Prophet()
model.fit(train_df)

# Create a dataframe for future predictions covering the test period
future = model.make_future_dataframe(periods=len(test_df), freq='D')
forecast = model.predict(future)

# Categorize forecasted daily values into quartiles based on the thresholds
forecast['quartile'] = pd.cut(forecast['yhat'], bins = [-np.inf] + list(quartiles) + [np.inf], labels=[1, 2, 3, 4])

# Extract the forecasted quartiles for the test period
forecasted_quartiles = forecast.iloc[-len(test_df):]['quartile'].astype(int)


# Categorize actual daily values in the test set into quartiles
test_df['quartile'] = pd.cut(test_df['y'], bins=[-np.inf] + list(quartiles) + [np.inf], labels=[1, 2, 3, 4])
actual_test_quartiles = test_df['quartile'].astype(int)


# Calculate the evaluation metrics
prophet_accuracy = accuracy_score(actual_test_quartiles, forecasted_quartiles)

# Print the evaluation metrics
print(f'Accuracy: {prophet_accuracy:.4f}')

10:43:21 - cmdstanpy - INFO - Chain [1] start processing
10:43:22 - cmdstanpy - INFO - Chain [1] done processing


Accuracy: 0.4249


In [33]:
# For illustrative purposes we show the quartiles for training data
train_df['quartile'] = pd.cut(train_df['y'], bins = [-np.inf] + list(quartiles) + [np.inf], labels=[1, 2, 3, 4])
train_df['quartile'].head()

0    2
1    3
2    3
3    3
4    2
Name: quartile, dtype: category
Categories (4, int64): [1 < 2 < 3 < 4]

## Convert time series data to tabular format through featurization

In [7]:
import tsfel
from sktime.transformations.panel.tsfresh import TSFreshFeatureExtractor

# Define tsfresh feature extractor
tsfresh_trafo = TSFreshFeatureExtractor(default_fc_parameters="minimal")

# Transform the training data using the feature extractor
X_train_transformed = tsfresh_trafo.fit_transform(X_train)

# Transform the test data using the same feature extractor
X_test_transformed = tsfresh_trafo.transform(X_test)

# Retrieves a pre-defined feature configuration file to extract all available features
cfg = tsfel.get_features_by_domain()

# Function to compute tsfel features per day
def compute_features(group):
    # TSFEL expects a DataFrame with the data in columns, so we transpose the input group
    features = tsfel.time_series_features_extractor(cfg, group, fs=1, verbose=0)
    return features


# Group by the 'day' level of the index and apply the feature computation
train_features_per_day = X_train.groupby(level='Date').apply(compute_features).reset_index(drop=True)
test_features_per_day = X_test.groupby(level='Date').apply(compute_features).reset_index(drop=True)

# Combine each featurization into a set of combined features for our train/test data
train_combined_df = pd.concat([X_train_transformed, train_features_per_day], axis=1)
test_combined_df = pd.concat([X_test_transformed, test_features_per_day], axis=1)

# Filter out features that are highly correlated with our target variable
column_of_interest = "PJME_MW__mean"
train_corr_matrix = train_combined_df.corr()
train_corr_with_interest = train_corr_matrix[column_of_interest]
null_corrs = pd.Series(train_corr_with_interest.isnull())
false_features = null_corrs[null_corrs].index.tolist()

columns_to_exclude = list(set(train_corr_with_interest[abs(train_corr_with_interest) > 0.8].index.tolist() + false_features))
columns_to_exclude.remove(column_of_interest)

# Filtered DataFrame excluding columns with high correlation to the column of interest
X_train_transformed = train_combined_df.drop(columns=columns_to_exclude)
X_test_transformed = test_combined_df.drop(columns=columns_to_exclude)

Feature Extraction: 100%|████████████████████████████████████████████████| 1205/1205 [00:00<00:00, 9625.59it/s]


In [8]:
# Define a function to classify each value into a quartile
def classify_into_quartile(value):
    if value < quartiles[0]:
        return 1  
    elif value < quartiles[1]:
        return 2  
    elif value < quartiles[2]:
        return 3  
    else:
        return 4  

y_train = X_train_transformed["PJME_MW__mean"]
X_train_transformed.drop("PJME_MW__mean", inplace=True, axis=1)

y_test = X_test_transformed["PJME_MW__mean"]
X_test_transformed.drop("PJME_MW__mean", inplace=True, axis=1)

energy_levels_train = y_train.apply(classify_into_quartile)
energy_levels_test = y_test.apply(classify_into_quartile)

In [34]:
X_train_transformed

,PJME_MW__standard_deviation,PJME_MW__variance,PJME_MW_Centroid,PJME_MW_Entropy,PJME_MW_FFT mean coefficient_0,PJME_MW_FFT mean coefficient_1,PJME_MW_FFT mean coefficient_10,PJME_MW_FFT mean coefficient_11,PJME_MW_FFT mean coefficient_12,PJME_MW_FFT mean coefficient_2,...,PJME_MW_Spectral roll-off,PJME_MW_Spectral skewness,PJME_MW_Spectral slope,PJME_MW_Spectral spread,PJME_MW_Spectral variation,PJME_MW_Standard deviation,PJME_MW_Variance,PJME_MW_Wavelet entropy,PJME_MW_Wavelet variance_4,PJME_MW_Wavelet variance_5
0,4097.961271,1.679329e+07,12.727435,1.0,5.207144e+06,1.736704e+08,45827.482018,79791.456899,56591.123457,1.758157e+08,...,0.083333,5.166394,-0.745390,0.050256,0.247164,4097.961271,1.679329e+07,1.896687,1.664806e+08,2.229225e+08
1,3718.008117,1.382358e+07,12.554067,1.0,3.425893e+06,1.564219e+08,65442.702058,94361.095951,20417.234568,1.455746e+08,...,0.083333,5.700734,-0.751053,0.046701,0.234079,3718.008117,1.382358e+07,1.895960,1.534287e+08,2.099539e+08
2,3241.304817,1.050606e+07,12.395692,1.0,2.600067e+06,1.015409e+08,1155.348647,71254.542792,19028.669753,1.227644e+08,...,0.083333,5.583426,-0.753413,0.044695,0.280831,3241.304817,1.050606e+07,1.895280,1.377279e+08,1.852873e+08
3,2259.371710,5.104761e+06,12.204000,1.0,4.219601e+04,6.502275e+07,35704.669659,92795.408167,11306.777778,5.256738e+07,...,0.083333,5.868139,-0.750643,0.052614,0.267369,2259.371710,5.104761e+06,1.898934,1.031623e+08,1.269314e+08
4,3250.463504,1.056551e+07,12.751234,1.0,7.678627e+05,2.307445e+08,8367.989724,2063.814511,1591.123457,3.033723e+07,...,0.041667,6.800978,-0.762032,0.036911,0.058819,3250.463504,1.056551e+07,1.893653,1.547531e+08,1.735752e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4812,2300.469354,5.292159e+06,12.546717,1.0,2.359637e+02,6.887002e+07,8707.188399,538.365570,3626.716049,5.525137e+07,...,0.083333,6.606409,-0.756860,0.043328,0.244304,2300.469354,5.292159e+06,1.896790,6.540976e+07,8.907364e+07
4813,1257.891390,1.582291e+06,11.779129,1.0,3.220031e+04,2.483169e+06,13221.386986,7869.500208,386.777778,3.375100e+07,...,0.083333,6.449819,-0.760264,0.046810,0.763902,1257.891390,1.582291e+06,1.893453,5.522637e+07,7.057513e+07
4814,2806.449903,7.876161e+06,12.418138,1.0,3.069504e+06,7.772943e+07,14016.952122,3739.317136,1671.901235,7.779717e+07,...,0.083333,5.875364,-0.753163,0.044136,0.221638,2806.449903,7.876161e+06,1.885767,8.673338e+07,1.304074e+08
4815,3302.024817,1.090337e+07,12.752696,1.0,2.520685e+06,1.790550e+08,3232.824925,11788.447837,3123.567901,6.768748e+07,...,0.083333,6.196339,-0.758445,0.037752,0.107072,3302.024817,1.090337e+07,1.886525,1.037424e+08,1.526816e+08


## Train and Evaluate GradientBoostingClassifier Model on multiclass tabular data

In [13]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(
    n_estimators=150,
    learning_rate=0.1,
    max_depth=4,
    min_samples_leaf=20,
    max_features='sqrt',
    subsample=0.8,
    random_state=42
)

gbc.fit(X_train_transformed, energy_levels_train)


y_pred_gbc = gbc.predict(X_test_transformed)
gbc_accuracy = accuracy_score(energy_levels_test, y_pred_gbc)
print(f'Accuracy: {gbc_accuracy:.4f}')

Accuracy: 0.8075


## Using AutoML to streamline things

Here’s all the code needed to train and deploy an AutoML supervised classifier:

In [ ]:
from cleanlab_studio import Studio

studio = Studio()
studio.create_project(
    dataset_id=energy_forecasting_dataset,
    project_name="ENERGY-LEVEL-FORECASTING",
    modality="tabular",
    task_type="multi-class",
    model_type="regular",
    label_column="daily_energy_level",
)

model = studio.get_model(energy_forecasting_model)
y_pred_automl = model.predict(test_data, return_pred_proba=True)

In [14]:
y_pred_automl_cleanlab = pd.read_csv("quartile-multiclass-pjme-testing-data_pred_probs.csv")
y_pred_automl_cleanlab = y_pred_automl_cleanlab["Suggested Label"]

In [15]:
automl_accuracy = accuracy_score(energy_levels_test, y_pred_automl_cleanlab)
print(f'Accuracy: {automl_accuracy:.4f}')

Accuracy: 0.8880


In [19]:
# Calculate error rates
error_rate_prophet = 1 - prophet_accuracy
error_rate_gbc = 1 - gbc_accuracy
error_rate_automl = 1 - automl_accuracy

print(f"Prophet error rate: {error_rate_prophet}")
print(f"GBC error rate: {error_rate_gbc}")
print(f"AutoML error rate: {error_rate_automl}")

# Calculate reduction in prediction error
error_reduction_gbc = error_rate_prophet - error_rate_gbc
error_reduction_automl_to_prophet = error_rate_prophet - error_rate_automl
error_reduction_automl_to_gbc = error_rate_gbc - error_rate_automl

# Convert error reduction to a percentage of improvement from one model to another
percentage_improvement_gbc = round((error_reduction_gbc / error_rate_prophet) * 100, 1)
percentage_improvement_automl_to_prophet = round((error_reduction_automl_to_prophet / error_rate_prophet) * 100, 1)
percentage_improvement_automl_to_gbc = round((error_reduction_automl_to_gbc / error_rate_gbc) * 100, 1)

print(f"GBC compared to Prophet resulted in a {percentage_improvement_gbc}% reduction in prediction error.")
print(f"AutoML compared to Prophet resulted in a {percentage_improvement_automl_to_prophet}% reduction in prediction error.")
print(f"AutoML compared to GBC resulted in a {percentage_improvement_automl_to_gbc}% reduction in prediction error.")

Prophet error rate: 0.5750825082508251
GBC error rate: 0.1925311203319502
AutoML error rate: 0.11203319502074693
GBC compared to Prophet resulted in a 66.5% reduction in prediction error.
AutoML compared to Prophet resulted in a 80.5% reduction in prediction error.
AutoML compared to GBC resulted in a 41.8% reduction in prediction error.
